In [1]:
import functools
import glob
import itertools as it
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#### Metrics

In [2]:
def precision_at_k(retrieved_doc_ids, relevant_doc_ids, k):
    rel_cnt = 0
    for doc_id in retrieved_doc_ids[:k]:
        if doc_id in relevant_doc_ids:
            rel_cnt += 1
    return np.array(rel_cnt / k)

def r_precision(retrieved_doc_ids, relevant_doc_ids):
    return precision_at_k(retrieved_doc_ids, relevant_doc_ids, len(relevant_doc_ids))

def average_precision(retrieved_doc_ids, relevant_doc_ids):
    precisions = []
    rel_cnt = 0
    for i, doc_id in enumerate(retrieved_doc_ids):
        if doc_id in relevant_doc_ids:
            rel_cnt += 1
            precisions.append(rel_cnt / (i + 1))
    return np.array(precisions).mean()

def mean_average_precision(retrieved_doc_ids, relevant_doc_ids):
    assert len(retrieved_doc_ids) == len(relevant_doc_ids)
    
    average_precisions = [average_precision(ret_ids, rel_ids) for ret_ids, rel_ids in 
                          zip(retrieved_doc_ids, relevant_doc_ids)]
    return np.array(average_precisions).mean()

precision_at_5 = functools.partial(precision_at_k, k=5)
precision_at_10 = functools.partial(precision_at_k, k=10)
precision_at_15 = functools.partial(precision_at_k, k=15)

#### Utility functions 

In [3]:
from IPython.display import HTML, display

def load_dataset(dirname):
    dataset_dict = {}
    files = glob.glob(f'{dirname}/**/*.json', recursive=True)
    for file in files:
        doc = Document.from_json(file)
        dataset_dict[doc.paper_id] = doc
    return dataset_dict

def append_score(d, results):
    for id, score in results:
        d[id].append(score)

def order_ids_by_score(ids, scores):
    score_order = scores.argsort()[::-1]
    return [ids[sc] for sc in score_order]

def eval_model(retrieved_ids, relevant_ids):
    return np.array([
        precision_at_5(retrieved_ids, relevant_ids),
        precision_at_10(retrieved_ids, relevant_ids),
        precision_at_15(retrieved_ids, relevant_ids),
        r_precision(retrieved_ids, relevant_ids),
        average_precision(retrieved_ids, relevant_ids),
    ])

def print_relevant(ranked_ids, relevant_ids):
    for i, id in enumerate(ranked_ids):
        pref = ''
        if id in relevant_ids:
            pref = '--> '
        print(f'{i+1}. {pref}{id}')

def print_dataframe(df):
    df_print = df.copy()
    df_print['Paper ID'] = df_print['Paper ID'].str.pad(60)
    return df_print.head()
    
    
def plot_metrics(models, metrics_matrix):
    assert len(models) == len(metrics_matrix)
    
    header = f"<tr>{''.join(f'<th>{metric}</th>' for metric in ['Model', 'P@5', 'P@10', 'P@15', 'R-prec', 'AveP'])}</tr>"
    rows = []
    for model, metrics in zip(models, metrics_matrix):
        metrics = [f'{metric:.2f}'for metric in metrics]
        row = f"<tr>{''.join(f'<td>{m}</td>' for m in [model, *metrics])}</tr>"
        rows.append(row)
        
    html = f"<table style='width:100%'>{header}{''.join(rows)}</table>"
    display(HTML(html))
    
def plot_map(models, map_scores):
    assert len(models) == len(map_scores)
    
    header = f"<tr>{''.join(f'<th>{metric}</th>' for metric in ['Model', 'MAP'])}</tr>"
    rows = []
    for model, map_score in zip(models, map_scores):
        row = f"<tr><td>{model}</td><td>{map_score:.2f}</td></tr>"
        rows.append(row)
    
    html = f"<table style='width:40%'>{header}{''.join(rows)}</table>"
    display(HTML(html))

In [4]:
class Document:
    
    def __init__(self, paper_id, title, abstract, body_text):
        self.paper_id = paper_id
        self.title = title
        self.abstract = abstract
        self.body_text = body_text
    
    @classmethod
    def from_json(cls, path):
        with open(path, 'r') as fd:
            data = json.load(fd)
        
        paper_id = data['paper_id']
        title = data['metadata']['title']
        abstract = '\n'.join([record['text'] for record in data['abstract']])
        body_text = '\n'.join([record['text'] for record in data['body_text']])
        return cls(paper_id, title, abstract, body_text)
    
    def __repr__(self):
        return f'{self.paper_id}: {self.title[:30]} ... {self.abstract[:200]} ... {self.body_text[:200]} ...'
        
    def _repr_html_(self):
        paper_html = f'<b>Paper ID:</b> {self.paper_id}'
        title_html = f'<h3>Title</h3> {self.title}'
        abstract_html = ['<p>' + record + '</p>' for record in self.abstract.split('\n')]
        abstract_html = '<h3>' + 'Abstract' + '</h3>' + ''.join(abstract_html)
        body_text_html = ['<p>' + record + '</p>' for record in self.body_text.split('\n')]
        body_text_html = '<h3>' + 'Body text' + '</h3>' + ''.join(body_text_html)  
        return paper_html + title_html + abstract_html + body_text_html

In [5]:
from dataset_utils import results2tuple, annotations2tuple

In [7]:
ANNOTATIONS_PATH = 'annotations-v3.txt'
annotations = annotations2tuple(ANNOTATIONS_PATH)
assert len(annotations) == 95

#### Transmission evaluation - What is known about Covid-19 transmission?

In [8]:
transmission_annotations = {annot[0]: [annot[1][0]] for annot in annotations}

append_score(transmission_annotations, results2tuple('results/lm-transmission-scores.txt'))
append_score(transmission_annotations, results2tuple('results/bm25-transmission-scores.txt'))
append_score(transmission_annotations, results2tuple('results/transmission-v2.txt')) 

In [9]:
transmission_flatten = [(k, *v) for k, v in transmission_annotations.items()]
transmission_df = pd.DataFrame(transmission_flatten, columns=['Paper ID', 'Relevance', 'LM', 'BM25', 'Our Model'])
print_dataframe(transmission_df)

,Paper ID,Relevance,LM,BM25,Our Model
0,15e842b8bf1a55c1df1c369a5f...,I,0.000000,0.459039,0.0
1,13f6bc615450f0470c2a265e0a...,I,0.360454,1.959558,0.0
2,ce660d55c0f3908969f3542d3c...,I,0.663033,2.413859,0.0
3,aae7699e079d2964df8078931c...,I,1.151933,3.219056,0.0
4,f72975ffd96b7ab1528afbea8f...,I,0.537950,2.258155,0.0


In [10]:
ids = transmission_df['Paper ID'].tolist()
relevant_ids_tran = transmission_df[transmission_df['Relevance'] == 'R']['Paper ID'].tolist()

ids_lm_tran = order_ids_by_score(ids, transmission_df['LM'].to_numpy())
ids_bm25_tran = order_ids_by_score(ids, transmission_df['BM25'].to_numpy())
ids_model_tran = order_ids_by_score(ids, transmission_df['Our Model'].to_numpy())

res_lm_tran = eval_model(ids_lm_tran, relevant_ids_tran)
res_bm25_tran = eval_model(ids_bm25_tran, relevant_ids_tran)
res_model_tran = eval_model(ids_model_tran, relevant_ids_tran)

plot_metrics(['LM', 'BM25', 'Our Model'], [res_lm_tran, res_bm25_tran, res_model_tran])
# relevant_ids_tran, ids_lm_tran[:15], ids_bm25_tran[:15], ids_model_tran[:15], ids_bm25_tran[9]

Model,P@5,P@10,P@15,R-prec,AveP
LM,0.80,0.60,0.47,0.67,0.69
BM25,0.80,0.70,0.47,0.78,0.70
Our Model,0.80,0.70,0.60,0.67,0.84


In [11]:
# print_relevant(ids_model_tran, relevant_ids_tran)

In [13]:
# print_relevant(ids_bm25_tran, relevant_ids_tran)

In [14]:
# print_relevant(ids_lm_tran, relevant_ids_tran)

#### Incubation evaluation - What is known about Covid-19 incubation?

In [15]:
incubation_annotations = {annot[0]: [annot[1][1]] for annot in annotations}

append_score(incubation_annotations, results2tuple('results/lm-incubation-scores.txt'))
append_score(incubation_annotations, results2tuple('results/bm25-incubation-scores.txt'))
append_score(incubation_annotations, results2tuple('results/incubation-v2.txt'))

In [16]:
incubation_flatten = [(k, *v) for k, v in incubation_annotations.items()]
incubation_df = pd.DataFrame(incubation_flatten, columns=['Paper ID', 'Relevance', 'LM', 'BM25', 'Our Model'])
print_dataframe(incubation_df)

,Paper ID,Relevance,LM,BM25,Our Model
0,15e842b8bf1a55c1df1c369a5f...,I,0.000000,0.459039,0.0
1,13f6bc615450f0470c2a265e0a...,I,0.360454,1.959558,0.0
2,ce660d55c0f3908969f3542d3c...,I,0.663033,2.413859,0.0
3,aae7699e079d2964df8078931c...,I,1.151933,3.219056,0.0
4,f72975ffd96b7ab1528afbea8f...,I,0.537950,2.258155,0.0


In [17]:
ids = incubation_df['Paper ID'].tolist()
relevant_ids_incub = incubation_df[incubation_df['Relevance'] == 'R']['Paper ID'].tolist()

ids_lm_incub = order_ids_by_score(ids, incubation_df['LM'].to_numpy())
ids_bm25_incub = order_ids_by_score(ids, incubation_df['BM25'].to_numpy())
ids_model_incub = order_ids_by_score(ids, incubation_df['Our Model'].to_numpy())

res_lm_incub = eval_model(ids_lm_incub, relevant_ids_incub)
res_bm25_incub = eval_model(ids_bm25_incub, relevant_ids_incub)
res_model_incub = eval_model(ids_model_incub, relevant_ids_incub)

plot_metrics(['LM', 'BM25', 'Our Model'], [res_lm_incub, res_bm25_incub, res_model_incub])
# relevant_ids_incub, ids_lm_incub[:5], ids_bm25_incub[:5], ids_model_incub[:5]

Model,P@5,P@10,P@15,R-prec,AveP
LM,0.60,0.50,0.33,0.60,0.64
BM25,0.40,0.50,0.33,0.40,0.55
Our Model,0.80,0.50,0.33,0.80,0.97


In [18]:
# print_relevant(ids_model_incub, relevant_ids_incub)

In [19]:
# print_relevant(ids_bm25_incub, relevant_ids_incub)

In [20]:
# print_relevant(ids_lm_incub, relevant_ids_incub)

In [22]:
map_lm = mean_average_precision([ids_lm_tran, ids_lm_incub], [relevant_ids_tran, relevant_ids_incub])
map_bm25 = mean_average_precision([ids_bm25_tran, ids_bm25_incub], [relevant_ids_tran, relevant_ids_incub])
map_model = mean_average_precision([ids_model_tran, ids_model_incub], [relevant_ids_tran, relevant_ids_incub])

plot_map(['LM', 'BM25', 'Our Model'], [map_lm, map_bm25, map_model])

Model,MAP
LM,0.66
BM25,0.62
Our Model,0.91


#### Pretty printing

In [23]:
from IPython.display import display
from ipywidgets import widgets

text = widgets.Text(placeholder='Enter Paper ID')
out = widgets.Output()

ids_dict = {
    'lm tran': ids_lm_tran,
    'bm25 tran': ids_bm25_tran,
    'model tran': ids_model_tran,
    'lm incub': ids_lm_incub,
    'bm25 incub': ids_bm25_incub,
    'model incub': ids_model_incub,
}
dataset_dict = load_dataset('dataset')

def handle_submit(sender):
    out.clear_output(wait=True)
    with out:
        paper_id = text.value
        if any(text.value.startswith(k) for k in ids_dict):
            key, ind = text.value.split(',')
            paper_id = ids_dict[key.strip()][int(ind.strip())]
        display(dataset_dict[paper_id])
        
display(text)
text.on_submit(handle_submit)

out

Text(value='', placeholder='Enter Paper ID')

Output()